<a href="https://colab.research.google.com/github/SilahicAmil/Intro-NLP/blob/main/Into_To_Natural_Language_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP fundamentals in Tensorflow

NLP has the goal of deriving information of of natural language (could be, sequences, text or speech)

Another common term for NLP provblems is sequence to sequence problem

## Check for GPU


In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-f0ce945a-f0d7-40c1-1c92-e8d614d29b29)


## Helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import a series of helper functions
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys


--2021-06-30 03:43:55--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-06-30 03:43:55 (63.5 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get our text data set

The data we're going to be using is Kaggle's intro to NLP dataset (text samples of Tweets labelled as a disaster or non disaster)

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

#Unzip data
unzip_data("nlp_getting_started.zip")

--2021-06-30 03:43:57--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.20.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2021-06-30 03:43:57 (148 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualize a text dataset

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
#Shuffle training data

train_df_shuffled = train_df.sample(frac=1,
                                    random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
# What does the test data look like?

test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
# How many examples of each class are there?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
len(train_df), len(test_df)

(7613, 3263)

In [9]:
#Visualize random training examples
import random
random_index = random.randint(0, len(train_df)-5) # Creat random indexes
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row

  print(f"Target: {target}", "(Real Disaster)" if target > 0 else "(Not a real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 1 (Real Disaster)
Text:
#BBSNews latest 4 #Palestine &amp; #Israel -  Six Palestinians Kidnapped in West Bank Hebron Home Demolished http://t.co/gne1fW0XHE

---

Target: 0 (Not a real disaster)
Text:
@VictoriaGittins what do you take me for I'm not a mass murderer! Just the one...

---

Target: 1 (Real Disaster)
Text:
Trafford Centre film fans angry after Odeon cinema evacuated following false fire alarm http://t.co/pFMn63VnAm http://t.co/vKwqbOJFJc

---

Target: 0 (Not a real disaster)
Text:
Sweater Stretcher http://t.co/naTz5iPV1x http://t.co/leaEBy6cR2

---

Target: 1 (Real Disaster)
Text:
Baltimore City : I-95 NORTH AT MP 54.8 (FORT MCHENRY TUNNEL BORE 3: Collision: I-95 NORTH AT MP 54.8 (FORT MCHENRY TUNNEL BORE 3 Nort...

---



## Split data into training and validation sets

In [10]:
from sklearn.model_selection import train_test_split


In [11]:
# Use train test split to split data into training and validation sets 
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [12]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [13]:
# Check the first 10 examples

train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers

When dealing with a text problem. One of the first things youll have to do before you can build the model is to convert the text to numbers

There are a few ways:
* Tokenization - Direct mapping of token
* Embedding- Create a matrix of feature vector for each token


## Text Vectorization (Tokenization)

In [14]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default Text Vectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # How many words in the vocabulary (Auto adds <OOV>)
                                       standardize="lower_and_strip_punctuation",
                                       split="whitespace",
                                       ngrams=None, # Create groups of n_words,
                                       output_mode="int", # How to map tokens to numbers
                                       output_sequence_length=None, # How long do you want the sequence to be
                                       pad_to_max_tokens=True)

In [15]:
# Find the average number of tokens (words) in the training tweets

round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
#Setup text vectorization variables
max_vocab_length =  10000 # Max # of words to have in vocab
max_length = 15 #Max lengeth sequence will be (average)

# Update vectorizer
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [18]:
# fit the text vectorizer to the training text

text_vectorizer.adapt(train_sentences)

In [21]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [25]:
# Choose a random sentence form the training dataset and tokenize it

random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
      \n\nVetorized Version:")

text_vectorizer([random_sentence])

Original text:
 Those that I have sworn to defend have proven themselves to be friends of the House Hailstorm.      

Vetorized Version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 161,   16,    8,   24, 7853,    5, 3065,   24, 9516, 2656,    5,
          21,  819,    6,    2]])>

In [31]:
# Get the unique words in the vocab
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words= words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 Most common words: {top_5_words}")
print(f"5 Least common words: {bottom_5_words}")


Number of words in vocab: 10000
5 Most common words: ['', '[UNK]', 'the', 'a', 'in']
5 Least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


## Creating an Embedding using an Emedding Layer

To make our embedding we are going to use TensorFlow's embedding layer

In [38]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # Input Shape
                             output_dim=128, # Anything divisble by 8 speeds up computing with machine learning
                             embeddings_initializer="uniform",
                             input_length=max_length)

embedding

In [45]:
# Get a random sentence from trainin set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
      n\nEmbedded Version:")

# Embed the random sentence

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Watch This Airport Get Swallowed Up By A Sandstorm In Under A Minute http://t.co/Z2Ph0ArzYI      n
Embedded Version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.04640846,  0.03199841,  0.02643691, ...,  0.02667708,
          0.02285974, -0.02035623],
        [-0.04244053, -0.01486727, -0.04917696, ...,  0.03240852,
          0.01755244, -0.02725856],
        [-0.0295953 , -0.01995931, -0.02846501, ...,  0.0480494 ,
         -0.02220783, -0.04424849],
        ...,
        [ 0.01514841,  0.02271548,  0.01507827, ...,  0.01657034,
          0.00401125, -0.02757516],
        [ 0.04292488,  0.00929397,  0.00269321, ..., -0.04860739,
         -0.04570031,  0.00020222],
        [-0.0128089 ,  0.01295615,  0.01649109, ..., -0.01546367,
          0.02149084,  0.03363803]]], dtype=float32)>

In [46]:
# Check out a single tokens embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.04640846,  0.03199841,  0.02643691, -0.01706852, -0.01665045,
        -0.0132269 ,  0.03955908,  0.00274519,  0.0104881 , -0.01990737,
         0.04903089,  0.02585319,  0.03698318, -0.04347626, -0.01152029,
         0.00466727, -0.04535225,  0.0265183 ,  0.0233996 , -0.00282496,
        -0.04701059, -0.01512676, -0.0181034 ,  0.02648312, -0.00599296,
         0.03264375,  0.00226791, -0.02698098,  0.04073068, -0.00044525,
        -0.02878445, -0.00090132, -0.00352639,  0.02816155, -0.02525281,
        -0.03293933,  0.0108333 , -0.0396922 ,  0.00708264,  0.03322795,
         0.04358241,  0.03594351,  0.0086114 ,  0.02474098,  0.03863012,
        -0.00049708,  0.00674769,  0.03242934, -0.03334029, -0.0497084 ,
        -0.04318304, -0.04431644, -0.00169207, -0.03295241, -0.03181844,
        -0.03120983,  0.03396286, -0.0336559 , -0.00460463,  0.0423357 ,
         0.03684772, -0.02290301, -0.04379764,  0.02080545,  0.01174276,
  

## Modelling a text data set (running experiments)

Now we've got a way to tunr our text sequences into numbers, it's time to start building a series of moedlling experiments.

We'll start with a baseline and move on from there.
* Model 0: Naive bayes (baseline) - SkLearn ML Map
* Model 1: Feed-forward nueral network (dense model)
* Model 2: LSTM Model (RNN)
* Model 3: GRU Model (RNN)
* Model 4: Bidrectional-LSTM Model (RNN)
* Model 5: 1D Covolutional Nueral Network (CNN)
* Model 6: TensorFlow Hub Pretrained Feature Extractor (Transfer learning for NLP)
* Model 8: Same as model 6 with 10% of training data

How are we going to approach all 8 models?

Using the standard steps in modelling is how we'll do it!
* Create our model
* Build a model
* Fit a model
* Evaluate our model
* Experiment 
* Save and Reload the model

